<a href="https://colab.research.google.com/github/gigaanderson/ETL/blob/main/ScoreInterno2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
# Função para obter a lista de IPs do CINSCORE
def get_cinscore_ips():

    # URL do CINSCORE para obter a lista de IPs
    url = "http://cinsscore.com/list/ci-badguys.txt"

    response = requests.get(url)
    if response.status_code == 200:
        # A lista de IPs está em formato de texto, cada linha contém um IP
        ips = [line.strip() for line in response.text.splitlines() if line and not line.startswith('#')]
        return ips[:10]  # Retorna apenas os 10 primeiros IPs
    else:
        print(f"Erro ao buscar IPs do CINSCORE: {response.status_code}")
        return []

In [3]:
# Função para enriquecer um IP usando a API do IPinfo.io
def enrich_ip(ip):
    url = 'https://api.abuseipdb.com/api/v2/check'
    API_KEY = 'ccfa49ac0340b951d1bec224ddb04d20a02e88212a2afa749294658a95f825a38802e3247b599dc9'

    querystring = {
        'ipAddress': ip,
        'maxAgeInDays': '90'  # Período de tempo para considerar relatórios (em dias)
    }
    headers = {
        'Accept': 'application/json',
        'Key': API_KEY
    }

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

In [4]:
# Função para calcular um score ponderado baseado na resposta da API
def calcular_score_proprio(resposta_abuseipdb):
    if resposta_abuseipdb is None:
        return 0

    data = resposta_abuseipdb.get('data', {})

    # Valores a serem utilizados no cálculo
    abuse_score = data.get('abuseConfidenceScore', 0)   # Normalizando para 0 a 1
    total_reports = min(data.get('totalReports', 0), 100)  # Normalizando e limitando
    distinct_users = min(data.get('numDistinctUsers', 0), 100) # Normalizando e limitando
    last_report = data.get('lastReportedAt', "")
    ip = data.get('ipAddress', 'N/A')  # IP address

    # Pesos para os fatores
    peso_abuse_score = 0.4     # Peso para o abuso
    peso_total_reports = 0.2    # Peso para o número de reports
    peso_distinct_users = 0.2   # Peso para a quantidade distinta de usuários
    peso_last_report = 0.2      # Peso para a data do último report

    # Cálculo do score da data do último report
    if last_report:
        last_report_date = datetime.strptime(last_report, "%Y-%m-%dT%H:%M:%S%z")
        days_since_last_report = (datetime.now(last_report_date.tzinfo) - last_report_date).days

        # Score baseado em quão recente é o último report
        score_last_report = max(0, (90 - days_since_last_report) / 90)
    else:
        score_last_report = 0

    # Normalizando o score do último report
    #score_last_report /= 100  # Para que fique entre 0 e 1

    # Média ponderada
    score_final = (
        (abuse_score * peso_abuse_score) +
        (total_reports * peso_total_reports) +
        (distinct_users * peso_distinct_users) +
        (score_last_report * peso_last_report)
    )

    # Arredondar o resultado final para 2 casas decimais
    score_final = round(score_final, 2)

    return ip, score_final, round(abuse_score, 2), round(distinct_users, 2), round(total_reports, 2), round(score_last_report, 2)


In [5]:
# Lista para armazenar os resultados
resultados = []

# Obter os 10 primeiros IPs do CINSCORE
ips = get_cinscore_ips()

# Para cada IP, buscar dados da API e calcular o score
for ip in ips:
    resposta_api = enrich_ip(ip)  # Chama a API para o IP
    if resposta_api:  # Certifica-se de que houve uma resposta
        resultados.append(calcular_score_proprio(resposta_api))

# Criar um DataFrame com os resultados
df = pd.DataFrame(resultados, columns=['IP', 'Score Interno', 'Abuse Score', 'Distinct Users', 'Total Reports', 'Score Last Report'])


# Configurar o Pandas para mostrar todas as colunas, sem truncamento
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.width', 1000)        # Definir a largura máxima da tabela (ajuste conforme necessário)
pd.set_option('display.max_colwidth', 100)  # Definir a largura máxima das colunas (ajuste conforme necessário)
pd.set_option('display.expand_frame_repr', False)  # Evitar que o DataFrame quebre em várias linhas

# Imprimir o DataFrame como tabela
print(df)

              IP  Score Interno  Abuse Score  Distinct Users  Total Reports  Score Last Report
0    1.15.118.23           66.6          100              32            100               1.00
1   1.158.95.124           13.0           27               4              6               0.98
2    1.20.102.56           72.4          100              61            100               1.00
3    1.209.45.24           72.8          100              63            100               1.00
4    1.212.6.106           70.2          100              50            100               1.00
5  1.220.149.234           69.4          100              46            100               1.00
6    1.221.74.88           12.6           26               5              5               1.00
7    1.226.61.41           80.0          100              99            100               1.00
8    1.226.83.54           67.0          100              34            100               1.00
9   1.227.83.100           75.4          100      

In [6]:
ips = get_cinscore_ips()

# Lista para armazenar os dados enriquecidos
enriched_ips = []

# Enriquecer cada IP usando a API do IPinfo.io
for ip in ips:
    enriched_data = enrich_ip(ip)
    if enriched_data:
        enriched_ips.append(enriched_data)

# Criar uma tabela (DataFrame) com os dados enriquecidos
df = pd.DataFrame(enriched_ips)

# Configurar o Pandas para mostrar todas as colunas, sem truncamento
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.width', 1000)        # Definir a largura máxima da tabela (ajuste conforme necessário)
pd.set_option('display.max_colwidth', 100)  # Definir a largura máxima das colunas (ajuste conforme necessário)
pd.set_option('display.expand_frame_repr', False)  # Evitar que o DataFrame quebre em várias linhas

# Mostrar a tabela com os dados
print(df)

                                                                                                  data
0  {'ipAddress': '1.15.118.23', 'isPublic': True, 'ipVersion': 4, 'isWhitelisted': False, 'abuseCon...
1  {'ipAddress': '1.158.95.124', 'isPublic': True, 'ipVersion': 4, 'isWhitelisted': False, 'abuseCo...
2  {'ipAddress': '1.20.102.56', 'isPublic': True, 'ipVersion': 4, 'isWhitelisted': False, 'abuseCon...
3  {'ipAddress': '1.209.45.24', 'isPublic': True, 'ipVersion': 4, 'isWhitelisted': False, 'abuseCon...
4  {'ipAddress': '1.212.6.106', 'isPublic': True, 'ipVersion': 4, 'isWhitelisted': False, 'abuseCon...
5  {'ipAddress': '1.220.149.234', 'isPublic': True, 'ipVersion': 4, 'isWhitelisted': False, 'abuseC...
6  {'ipAddress': '1.221.74.88', 'isPublic': True, 'ipVersion': 4, 'isWhitelisted': False, 'abuseCon...
7  {'ipAddress': '1.226.61.41', 'isPublic': True, 'ipVersion': 4, 'isWhitelisted': False, 'abuseCon...
8  {'ipAddress': '1.226.83.54', 'isPublic': True, 'ipVersion': 4, 'isWhit